Going on vaction  https://github.com/AnnaNican/optimizers/blob/master/GoingOnVacation.ipynb

Teacher classes assignment https://github.com/UBC-MDS/DSCI_512_alg-data-struct/blob/master/lectures/lecture7.ipynb

In [1]:
import pandas as pd
import numpy as np
import re

from pulp import *


import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [2]:
#%pwd
!pip install pulp

In [ ]:
df = pd.read_csv("/content/my_data/appartment optimizaton.csv", sep = ",")
df
#df.dtypes

,index,address,nr_room,size,floor_nr,toilet_window,sub_to_centre_min,cost,monthly_payment,driftkost,ad_reading_nr,bus,metro,simhall,build_year,ad_exist_days,price_raise_6m,price_raise_12m,price_raise_36m
0,1,farsta,2.5,63.5,6,10,40,1795000,4650,659,3307,0,1,1,0,12,0.07,0.15,0.10
1,2,järfälla_jakobsberg,2.0,65.5,3,1,30,1995000,4142,400,860,0,1,1,0,5,0.07,0.07,0.05
2,3,salem,2.0,65.0,1,1,50,1975000,4556,450,1528,0,1,1,1,11,-0.01,0.05,0.04


In [ ]:
# create the LP object, set up as a minimization problem --> since we want to minimize the costs 
prob = pulp.LpProblem('ChooseFlat', LpMinimize)

In [ ]:
# create decision variables

decision_variables = []
for rownum, row in df.iterrows():
	variable = str('x' + str(rownum))
	variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
	decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))
print ("Array with Decision Variables:" + str(decision_variables))

Total number of decision_variables: 3
Array with Decision Variables:[x0, x1, x2]


In [ ]:
# define objective function: minimize the cost

total_cost = ""
for rownum, row in df.iterrows():
	for i, schedule in enumerate(decision_variables):
		if rownum == i:
			formula = (((row['cost'] + row['monthly_payment'] + row['driftkost'])/row['size']) + row['price_raise_36m']*(-100) + row['price_raise_6m']*(-10))* row['toilet_window']*row['floor_nr']*schedule
			total_cost += formula

prob += total_cost
print ("Optimization function: " + str(total_cost))




Optimization function: 1700437.3700787402*x0 + 91564.97633587787*x1 + 30457.73076923077*x2


In [ ]:
transport_time = 30 # minute
total_time_min = ""
for rownum, row in df.iterrows():
	for i, schedule in enumerate(decision_variables):
		if rownum == i:
			formula = (row['metro'] + row['bus']) * row['sub_to_centre_min'] *schedule
			total_time_min += formula

prob += (total_time_min <= transport_time)
prob

ChooseFlat:
MINIMIZE
1700437.3700787402*x0 + 91564.97633587787*x1 + 30457.73076923077*x2 + 0.0
SUBJECT TO
_C1: 40 x0 + 30 x1 + 50 x2 <= 30

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer

In [ ]:
# room, toilet  window: to get average point in order to compare different flats. mean(# of rooms) = 2, mean(toilet window) = 0.5, mean(# of floors in a buildning) = 3

our_expect = 1.5 # 
total_points = ""
for rownum, row in df.iterrows():
	for i, schedule in enumerate(decision_variables):
		if rownum == i:
			formula = (((row['nr_room'] + row['toilet_window'] + row['floor_nr'])/(2+0.5+3)) + row['simhall'])*schedule
			total_points += formula

prob += (total_points >= our_expect)
prob

ChooseFlat:
MINIMIZE
1700437.3700787402*x0 + 91564.97633587787*x1 + 30457.73076923077*x2 + 0.0
SUBJECT TO
_C1: 40 x0 + 30 x1 + 50 x2 <= 30

_C2: 4.36363636364 x0 + 2.09090909091 x1 + 1.72727272727 x2 >= 1.5

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer

In [ ]:
print(prob)
prob.writeLP("ChooseFlat.lp" )

ChooseFlat:
MINIMIZE
1700437.3700787402*x0 + 91564.97633587787*x1 + 30457.73076923077*x2 + 0.0
SUBJECT TO
_C1: 40 x0 + 30 x1 + 50 x2 <= 30

_C2: 4.36363636364 x0 + 2.09090909091 x1 + 1.72727272727 x2 >= 1.5

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer



[x0, x1, x2]

In [ ]:
optimization_result = prob.solve()

#assert optimization_result == pulp.LpStatusOptimal
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
	print(v.name, "=", v.varValue)

Status: Optimal
Optimal Solution to the problem:  91564.97633587787
Individual decision_variables: 
x0 = 0.0
x1 = 1.0
x2 = 0.0


In [ ]:
variable_name = []
variable_value = []

for v in prob.variables():
	variable_name.append(v.name)
	variable_value.append(v.varValue)

opt_res = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in opt_res.iterrows():
	value = re.findall(r'(\d+)', row['variable'])
	opt_res.loc[rownum, 'variable'] = int(value[0])

opt_res = opt_res.sort_values(by = 'variable')

#append results
for rownum, row in df.iterrows():
	for results_rownum, results_row in opt_res.iterrows():
		if rownum == results_row['variable']:
			df.loc[rownum, 'decision'] = results_row['value']
            
df

,index,address,nr_room,size,floor_nr,toilet_window,sub_to_centre_min,cost,monthly_payment,driftkost,ad_reading_nr,bus,metro,simhall,build_year,ad_exist_days,price_raise_6m,price_raise_12m,price_raise_36m,decision
0,1,farsta,2.5,63.5,6,10,40,1795000,4650,659,3307,0,1,1,0,12,0.07,0.15,0.10,0.0
1,2,järfälla_jakobsberg,2.0,65.5,3,1,30,1995000,4142,400,860,0,1,1,0,5,0.07,0.07,0.05,1.0
2,3,salem,2.0,65.0,1,1,50,1975000,4556,450,1528,0,1,1,1,11,-0.01,0.05,0.04,0.0


In [ ]:
df[df['decision'] == 1]

,index,address,nr_room,size,floor_nr,toilet_window,sub_to_centre_min,cost,monthly_payment,driftkost,ad_reading_nr,bus,metro,simhall,build_year,ad_exist_days,price_raise_6m,price_raise_12m,price_raise_36m,decision
1,2,järfälla_jakobsberg,2.0,65.5,3,1,30,1995000,4142,400,860,0,1,1,0,5,0.07,0.07,0.05,1.0


Minimize Cost 

In [2]:
data = {"location": ["Linköping", "Stockholm"],
        "köp": [1700000, 1700000],
        "hyra": [12000, 15000],
        "månadskostnad": [7900, 6800],
        "storlek": [48, 40],
        "trafik_kostnad": [5000, 1000]}
df = pd.DataFrame(data)
#df.index = ["linköping", "stockholm"]
df

,location,köp,hyra,månadskostnad,storlek,trafik_kostnad
0,Linköping,1700000,12000,7900,48,5000
1,Stockholm,1700000,15000,6800,40,1000


In [3]:
# create the LP object, set up as a minimization problem --> since we want to minimize the costs 
prob = pulp.LpProblem('Minimize cost', LpMinimize)

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
locations = ["Linköping", "Stockholm"]
buy_rent = ["buy", "rent"]

In [5]:
location_choice = LpVariable.dicts('location choice',
                                   ((i, j) for i in locations for j in buy_rent),
                                   lowBound = 0, upBound = 1,
                                   cat = 'Integer')
location_choice

{('Linköping', 'buy'): location_choice_('Linköping',_'buy'),
 ('Linköping', 'rent'): location_choice_('Linköping',_'rent'),
 ('Stockholm', 'buy'): location_choice_('Stockholm',_'buy'),
 ('Stockholm', 'rent'): location_choice_('Stockholm',_'rent')}

In [6]:
for rownum, row in df.iterrows():
  prob += lpSum([row['hyra']*location_choice[(i, j)] + row['månadskostnad']*location_choice[(i, j)] + row['trafik_kostnad']*location_choice[(i, j)] 
                       for i in locations for j in buy_rent])
  
print(prob)

Minimize_cost:
MINIMIZE
22800*location_choice_('Linköping',_'buy') + 22800*location_choice_('Linköping',_'rent') + 22800*location_choice_('Stockholm',_'buy') + 22800*location_choice_('Stockholm',_'rent') + 0
VARIABLES
0 <= location_choice_('Linköping',_'buy') <= 1 Integer
0 <= location_choice_('Linköping',_'rent') <= 1 Integer
0 <= location_choice_('Stockholm',_'buy') <= 1 Integer
0 <= location_choice_('Stockholm',_'rent') <= 1 Integer



/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1537: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [7]:
for rownum, row in df.iterrows():
  prob += lpSum([row['hyra']*location_choice[i, "rent"] + row['trafik_kostnad']*location_choice[i, "rent"] for i in locations]) <= 13000 
print(prob)

Minimize_cost:
MINIMIZE
22800*location_choice_('Linköping',_'buy') + 22800*location_choice_('Linköping',_'rent') + 22800*location_choice_('Stockholm',_'buy') + 22800*location_choice_('Stockholm',_'rent') + 0
SUBJECT TO
_C1: 17000 location_choice_('Linköping',_'rent')
 + 17000 location_choice_('Stockholm',_'rent') <= 13000

_C2: 16000 location_choice_('Linköping',_'rent')
 + 16000 location_choice_('Stockholm',_'rent') <= 13000

VARIABLES
0 <= location_choice_('Linköping',_'buy') <= 1 Integer
0 <= location_choice_('Linköping',_'rent') <= 1 Integer
0 <= location_choice_('Stockholm',_'buy') <= 1 Integer
0 <= location_choice_('Stockholm',_'rent') <= 1 Integer



In [7]:
decision_variables = []
for rownum, row in df.iterrows():
	variable = str('x' + str(rownum))
	variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
	decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))
print ("Array with Decision Variables:" + str(decision_variables))

Total number of decision_variables: 2
Array with Decision Variables:[x0, x1]


In [8]:
# objective function
total_cost = ""
for rownum, row in df.iterrows():
	for i, location in enumerate(decision_variables):
		if rownum == i:
			formula = row['hyra']*location + row['månadskostnad']*location + row['trafik_kostnad']*location
			total_cost += formula

prob += total_cost
print ("Optimization function: " + str(total_cost))

Optimization function: 24900*x0 + 22800*x1


In [9]:
budget = 13000
total_cost_estimate = ""
for rownum, row in df.iterrows():
	for i, location in enumerate(decision_variables):
		if rownum == i:
			formula = row['hyra']*location + row['månadskostnad']*location + row['trafik_kostnad']*location
			total_cost_estimate += formula

prob += (total_cost_estimate <= budget)

In [ ]:
budge = 13000

for location in locations:
  for choice in buy_rent:
    if choice == "buy":
      # print(location)
      # print(choice)
      #print(x[location][choice] <= budget)
      prob += (x[location][choice] <= budget)

In [16]:
prob

Minimize_cost:
MINIMIZE
49800*x0 + 45600*x1 + 0
SUBJECT TO
_C1: 238500 x1 <= 13000

_C2: 24900 x0 + 22800 x1 <= 13000

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer

In [ ]:
locations = df.index.values.tolist()
locations

['linköping', 'stockholm']

In [ ]:
buy_rent = ["buy", "rent"]

In [ ]:
# create decision variables
x = LpVariable.dicts("x", (locations, buy_rent), 0, 1, LpInteger)
x

{'linköping': {'buy': x_linköping_buy, 'rent': x_linköping_rent},
 'stockholm': {'buy': x_stockholm_buy, 'rent': x_stockholm_rent}}

In [ ]:
location_buy_rent = 1
for choice in buy_rent:
    prob += (lpSum(x[location][choice] for location in locations) == location_buy_rent )
# for location in locations:
#     prob += (lpSum(x[choice][location] for choice in buy_rent) == location_buy_rent )

In [15]:
print(prob)

Minimize_cost:
MINIMIZE
49800*x0 + 45600*x1 + 0
SUBJECT TO
_C1: 238500 x1 <= 13000

_C2: 24900 x0 + 22800 x1 <= 13000

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer



In [ ]:
budget = 1800000
for location in locations:
  for choice in buy_rent:
      # print(location)
      # print(choice)
      print(x[location][choice] <= budget)

x_linköping_buy <= 1800000
x_linköping_rent <= 1800000
x_stockholm_buy <= 1800000
x_stockholm_rent <= 1800000


In [ ]:
objective_terms = list()
for location in locations:
  for choice in buy_rent:
      # print(location)
      # print(choice)
      objective_terms.append(x[location][choice])
prob += pulp.lpSum(objective_terms)

In [ ]:
print(prob)

Minimize_cost:
MINIMIZE
1*x_linköping_buy + 1*x_linköping_rent + 1*x_stockholm_buy + 1*x_stockholm_rent + 0
SUBJECT TO
_C1: x_linköping_buy + x_stockholm_buy = 1

_C2: x_linköping_rent + x_stockholm_rent = 1

VARIABLES
0 <= x_linköping_buy <= 1 Integer
0 <= x_linköping_rent <= 1 Integer
0 <= x_stockholm_buy <= 1 Integer
0 <= x_stockholm_rent <= 1 Integer



In [ ]:
budge = 1800000

for location in locations:
  for choice in buy_rent:
    if choice == "buy":
      # print(location)
      # print(choice)
      #print(x[location][choice] <= budget)
      prob += (x[location][choice] <= budget)

In [11]:
print(prob)

Minimize_cost:
MINIMIZE
24900*x0 + 22800*x1 + 0
SUBJECT TO
_C1: 24900 x0 + 22800 x1 <= 13000

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer



In [8]:
prob.solve()
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
	print(v.name, "=", v.varValue)

Status: Optimal
Optimal Solution to the problem:  0.0
Individual decision_variables: 
location_choice_('Linköping',_'buy') = 0.0
location_choice_('Linköping',_'rent') = 0.0
location_choice_('Stockholm',_'buy') = 0.0
location_choice_('Stockholm',_'rent') = 0.0
